In [86]:
# Import pandas for data manipulation and analysis
import pandas as pd

# Import regular expressions for string manipulation
import re

# Import Counter from collections for counting occurrences
from collections import Counter

# Import matplotlib.pyplot for data visualization
import matplotlib.pyplot as plt

# Import seaborn for advanced statistical data visualization
import seaborn as sns

from datetime import datetime


In [87]:
# Load the chat file


In [88]:
import re
from datetime import datetime
import pandas as pd

def clean_text(text):
    """Remove invisible Unicode characters like the Left-to-Right Mark."""
    return re.sub(r'[\u200e\u200f]', '', text)

def parse_user_message(line):
    # Regular expression to capture date, time, author, and message for standard messages
    regex_pattern = r"^\[(\d{1,2}/\d{1,2}/\d{4}), (\d{2}:\d{2}:\d{2})\] ([^:]+): (.*)$"
    
    match = re.match(regex_pattern, clean_text(line))
    if match:
        date = match.group(1)
        time = match.group(2)
        author = match.group(3)
        message = match.group(4)
    elif "omitted" in line:
        parts = clean_text(line).split('] ', 1)
        datetime_part = parts[0].strip('[')
        rest = parts[1] if len(parts) > 1 else ""
        author_message = rest.split(': ', 1)
        author = author_message[0] if len(author_message) > 1 else "Unknown"
        message = "Media Omitted"
        date, time = datetime_part.split(', ')
    else:
        return None
    
    date_time_str = f"{date}, {time}"
    # Using clean_text to ensure no invisible characters are present
    date_time = datetime.strptime(clean_text(date_time_str), '%d/%m/%Y, %H:%M:%S')
    return {'DateTime': date_time, 'Author': author, 'Message': message}

# Assuming you are reading the chat file as before
with open('data/_chat.txt', encoding='utf-8') as file:
    lines = file.readlines()

# Parse lines and filter out None values
parsed_data = filter(None, (parse_user_message(line) for line in lines))

# Convert to a DataFrame
df = pd.DataFrame(list(parsed_data))

# Display the DataFrame or proceed with further analysis
print(df.head(50))


              DateTime               Author  \
0  2020-11-18 21:47:16   RMS DIGITAL REVAMP   
1  2020-11-18 21:47:16        Carol Wanyama   
2  2022-10-19 14:00:38   RMS DIGITAL REVAMP   
3  2022-10-19 14:04:16        Carol Wanyama   
4  2022-10-19 14:04:26        Carol Wanyama   
5  2022-10-19 14:05:24       Wachira Waruru   
6  2022-10-19 14:07:39        Carol Wanyama   
7  2022-10-19 14:08:30  Crispin Royal Media   
8  2022-10-19 14:08:44        Carol Wanyama   
9  2022-10-19 14:10:28       Wachira Waruru   
10 2022-10-19 14:10:37        Wahiga Mwaura   
11 2022-10-19 14:11:23        Carol Wanyama   
12 2022-10-19 14:16:08          Ephy Mwangi   
13 2022-10-19 14:17:31     Alex Murungi Rms   
14 2022-10-19 14:18:24       Wachira Waruru   
15 2022-10-19 14:18:31       Wachira Waruru   
16 2022-10-19 14:18:51        Carol Wanyama   
17 2022-10-19 14:19:51               Latifa   
18 2022-10-19 14:22:57  Crispin Royal Media   
19 2022-10-19 14:23:44          Ephy Mwangi   
20 2022-10-19

#### Total Messages, Media, and Members

In [92]:
media_messages = df[df['Message'] == 'image omitted']
print(f"Total Imgae Media Messages: {len(media_messages)}")


Total Imgae Media Messages: 1161


In [94]:
# Define a list of patterns that WhatsApp uses for omitted media messages
media_patterns = [
    'image omitted',
    'video omitted',
    'GIF omitted',
    'audio omitted',
    'document omitted',
    'sticker omitted'
]

# Combine the patterns into a single regex expression
media_regex = '|'.join([f"\\b{pattern}\\b" for pattern in media_patterns])

# Filter messages containing any of the media patterns and count them
media_messages_count = df[df['Message'].str.contains(media_regex, case=False, na=False)].shape[0]

print(f"Total Media Messages: {media_messages_count}")



Total Media Messages: 1650


In [96]:
total_messages = len(df)
total_members = df['Author'].nunique()

print(f"Total Messages: {total_messages}")
print(f"Total Members: {total_members}")



Total Messages: 6038
Total Members: 25
